In [33]:
import pandas as pd
import geopandas as gpd
from p_tqdm import p_map,p_umap,p_uimap,p_imap


import glob, re , os 
import numpy as np
dloc = '2021-oa-data'
oloc = '../static/'
tloc = 'temp/'

In [2]:
# number of points 
# total = pd.read_csv(dloc+'/usual-residents-oa.csv').Count.sum()
# total
# waterways 
# water = gpd.read_file('inland-water.geojson')

In [3]:
geom =  gpd.read_file('Output_Areas_(Dec_2021)_Boundaries_Full_Clipped_EW_(BFC).geojson').set_index('OA21CD')
geom = geom.geometry

In [4]:
import random
from shapely.geometry import Point

def generate_random(number, polygon):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return points

In [7]:
def makepoints(area_chunks):
    '''
    This functions processes the points for several areas'''
    dummy = []

    for area in area_chunks:
        row = data.loc[area]

        for col in selection:
            pts = generate_random(int(row[col]), geom.loc[area])
            #need a format we can turn into geojson
            dummy.extend([[area,col,p] for p in pts])

    return dummy



In [5]:
type = 'TS021_ethnic_group_tb_6a_2021'

data = pd.read_csv(dloc+'/'+type+'.csv').set_index('Geography code')

In [6]:
start = list(data.columns).index('Classification') + 1
end = list(data.columns).index('Total')
selection = list(data.columns)[start:end]
selection

data = data[selection]

In [8]:
# london = open('south.txt').read().split(',')
london = list(set(data.index) & set(geom.index))
len(london), len(data), len(london)/len(data)

(188880, 232183, 0.813496250802169)

In [9]:
allpoints = []
nit = 7000
ncores = 8

allpoints = []
# ratio optimised for macbook pro 8c 32gb
for res in p_uimap(makepoints,np.array_split(london,nit),num_cpus=ncores):
    allpoints.extend(res)

    


100%|██████████| 7000/7000 [29:55<00:00,  3.90it/s]  


In [10]:
import pickle

with open('temp.pickle', 'wb') as handle:
    pickle.dump(allpoints, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('temp.pickle', 'rb') as handle:
#     allpoints = pickle.load(handle)

In [13]:

gpd.GeoDataFrame(allpoints[:3],columns='cd cat geometry'.split()).to_file('sample.geojson', driver="GeoJSON")  

/Users/danielellis/opt/miniconda3/envs/DataServer/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [80]:
f = open(tloc + type + '.geojson','w')



f.write('''{
"type": "FeatureCollection",
"features": [
''')



def linewise(segment):
    fstr = ''
    for x in segment:
        p = x[2].xy
        fstr+= '''{ "type": "Feature", "properties": { "cd": "%s", "cat": "%s" }, "geometry": { "type": "Point", "coordinates": [ %0.6f, %0.6f ] } },\n'''%(x[0],x[1],p[0][0],p[1][0])
    return fstr

g = len(allpoints)//50000
split = [allpoints[i:i+g]for i in range(0,len(allpoints),g) ]


for res in p_uimap(linewise, split, num_cpus=ncores):
    f.write(res)

f.write('{  }\n]}')

f.close()



  0%|          | 0/50041 [00:00<?, ?it/s]

In [79]:
len(allpoints)//50000

1191

595.98449


In [32]:

# def save_gj(x):
#     fn, dummy = x
#     pgdf = gpd.GeoDataFrame(dummy,columns='cd cat geometry'.split())
#     geo = tloc + type + '_%02d.geojson'%fn
#     # files.append(geo)
#     pgdf.to_file(geo, driver="GeoJSON")  
#     return geo


# os.system(f'rm {tloc}*.geojson')
# p_umap(save_gj, enumerate(np.array_split(allpoints,10)),num_cpus=ncores)



In [58]:
# use parallel! as newline delimited geojson
'-z16 -Bg'
os.system(f'rm -rf {oloc}/{type}')
os.system(f'/usr/local/bin/tippecanoe -f -l {type} -r3 --drop-fraction-as-needed --no-tile-compression --no-tile-size-limit --no-feature-limit -P --output-to-directory="{oloc}/{type}" -zg -B12 -Z9 {tloc}*')



temp/TS021_ethnic_group_tb_6a_2021.geojson:3: Found ] at top level
temp/TS021_ethnic_group_tb_6a_2021.geojson:4: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
10 features, 150 bytes of geometry, 16 bytes of separate metadata, 296 bytes of string pool
Choosing a maxzoom of -z11 for features typically 442 feet (135 meters) apart, and at least 51 feet (16 meters) apart
  99.5%  11/1016/650  


0

In [ ]:
st='' 
key = '`'
for i,s in enumerate(selection):
    st+= f'"{s}" , colour[{i}],'
    key+="<span style='color:${colour[%d]}'> %s </span><br>"%(i,s)
st+='"#ccc"'
key+='`'

print(st)
print()
print(key)


In [ ]:
selection